In [1]:
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='Constant_forcing', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/saildrone_southern_ocean/gotm/code
   gotmdir_data: /Users/qingli/work/saildrone_southern_ocean/gotm/data
  gotmdir_build: /Users/qingli/work/saildrone_southern_ocean/gotm/build
    gotmdir_exe: /Users/qingli/work/saildrone_southern_ocean/gotm/exe
    gotmdir_run: /Users/qingli/work/saildrone_southern_ocean/gotm/run
 gotmdir_figure: /Users/qingli/work/saildrone_southern_ocean/gotm/figure
   gotmdir_tool: /Users/qingli/work/saildrone_southern_ocean/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 3 ms, sys: 11.2 ms, total: 14.2 ms
Wall time: 355 ms


## Configuration
Initialize the GOTM configuration

In [8]:
cfg = m.init_config()

 ------------------------------------------------------------------------
 GOTM started on 2021/09/16 at 15:53:48
 ------------------------------------------------------------------------
    initialize_gotm
 ------------------------------------------------------------------------
        Reading configuration from: gotm.yaml
        configuring modules ....
    init_airsea_yaml
        done
    init_observations_yaml
    init_stokes_drift_yaml
        done
    init_turbulence_yaml
        done.
    init_cvmix_yaml
        done.
    init_meanflow_yaml
        done
    init_eqstate_yaml
        done.
 Your configuration has been written to /Users/qingli/work/saildrone_southern_ocean/gotm/run/Constant_forcing/gotm.yaml.
STOP 0


Generating default configuration at '/Users/qingli/work/saildrone_southern_ocean/gotm/run/Constant_forcing/gotm.yaml'...
Done!


Update the configuration

In [9]:
# setup
title = 'Constant forcing'
nlev = 300
cfg['title'] = title
cfg['location']['name'] = 'Idealized'
cfg['location']['latitude'] = -53.15
cfg['location']['longitude'] = -142.9
cfg['location']['depth'] = 300
cfg['time']['start'] = '2019-02-09 00:00:00'
cfg['time']['stop']  = '2019-02-14 00:00:00'
cfg['time']['dt']    = 600.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 3
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = os.getcwd()
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = datadir+'/t_prof.dat'
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = datadir+'/s_prof.dat'
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['heat']['constant_value'] = -300
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['constant_value'] = 0.67
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['method'] = 'constant'
cfg['surface']['swr']['constant_value'] = 150
cfg['surface']['precip']['method'] = 'constant'
cfg['waves']['stokes_drift']['us']['method'] = 'off'
cfg['waves']['stokes_drift']['vs']['method'] = 'off'

# EOS -- use linear
cfg['eq_state']['form'] = 'full-pot'

# water type (Jerlov IB)
cfg['light_extinction']['method'] = 'jerlov-ib'

In [10]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

In [11]:
# collect the configurations and the labels of the two runs
cfgs = []
labels = []

In [12]:
sw_list = {'group1': 524.3,
           'group2': 103.2}
lw_list = {'group1': -43.6,
           'group2': -41}
shf_list = {'case1': 17.0,
            'case2': 32.0,
            'case3': 2.0}
lhf_list = {'case1': 61.0,
            'case2': 87.0,
            'case3': 35.0}
for gkey in sw_list.keys():
    for ckey in shf_list.keys():
        rlabel = '_'.join([gkey, ckey])
        print(rlabel)
        cfg['turbulence']['turb_method'] = 'second_order' 
        cfg['surface']['swr']['constant_value'] = sw_list[gkey]
        cfg['surface']['fluxes']['heat']['constant_value'] = lw_list[gkey]+shf_list[ckey]+lhf_list[ckey]
        cfgs.append(copy.deepcopy(cfg))
        labels.append(rlabel+'_GLS-C01A')
        cfg['turbulence']['turb_method'] = 'cvmix'
        cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
        cfgs.append(copy.deepcopy(cfg))
        labels.append(rlabel+'_KPP-CVMix')

group1_case1
group1_case2
group1_case3
group2_case1
group2_case2
group2_case3


## Run the model

In [13]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 39.3 ms, sys: 30.6 ms, total: 69.9 ms
Wall time: 3.49 s
